# Problem Set 1
## Members
    - Edmundo Arias – 202110688
    - Lucia Maldonado –
    - Juan Diego Heredia –

### Date: September something, 2024

# 0. Set Up

In [19]:
# nolint start
# clear environment and load libraries
rm(list = ls())
# nolint start

# install and load required packages
packages <- c(
    "tidyverse", "ggplot2", "stargazer",
    "rvest", "dplyr", "httr", "boot", "broom",
    "lmtest", "fixest", "sandwich", "lmtest", 
    "parallel", "foreach", "doParallel",
    "future.apply"
)
invisible(lapply(packages, function(pkg) {
    if (!requireNamespace(pkg, quietly = TRUE)) install.packages(pkg)
    library(pkg, character.only = TRUE)
}))

Loading required package: future



# 1. Data

In [20]:
plan(multisession) # Paralellizing

# Define the scraping function
scrape_page <- function(url) {
  response <- GET(url)
  if (status_code(response) != 200) {
    warning(paste("Failed to retrieve page:", url)) 
    return(NULL) 
  }
  content <- content(response, "text")
  page <- read_html(content)
  include_element <- html_node(page, "[w3-include-html]")
  include_url <- html_attr(include_element, "w3-include-html")
  full_include_url <- paste0("https://ignaciomsarmiento.github.io/GEIH2018_sample/", include_url)
  included_content <- read_html(full_include_url)
  df_page <- html_node(included_content, "table") %>% html_table()
  return(df_page)
}

In [21]:
base_url <- "https://ignaciomsarmiento.github.io/GEIH2018_sample/page"

# Use future_lapply to run tasks asynchronously -> gracias Ignacio :)
df_list <- future_lapply(1:10, function(i) {
  url <- paste0(base_url, i, ".html")
  df_page <- scrape_page(url)
  if (!is.null(df_page)) print(paste("Successfully scraped page", i))  # Print status if successful
  return(df_page)
})
df_list <- Filter(Negate(is.null), df_list)

# combine
df <- do.call(rbind, df_list)

[1] "Successfully scraped page 1"
[1] "Successfully scraped page 2"
[1] "Successfully scraped page 3"
[1] "Successfully scraped page 4"
[1] "Successfully scraped page 5"
[1] "Successfully scraped page 6"
[1] "Successfully scraped page 7"
[1] "Successfully scraped page 8"
[1] "Successfully scraped page 9"
[1] "Successfully scraped page 10"


In [22]:
df

Var.1,directorio,secuencia_p,orden,clase,dominio,mes,estrato1,sex,age,⋯,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4514331,1,2,1,BOGOTA,1,2,0,29,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,4514331,1,1,1,BOGOTA,1,2,1,36,⋯,NA,NA,NA,8404.320,NA,NA,NA,NA,1620833,8404.320
3,4514332,1,4,1,BOGOTA,1,2,1,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,4514332,1,3,1,BOGOTA,1,2,1,7,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,4514332,1,1,1,BOGOTA,1,2,0,32,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,4514332,1,2,1,BOGOTA,1,2,1,35,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,4514333,1,4,1,BOGOTA,1,2,1,18,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,4514333,1,2,1,BOGOTA,1,2,0,49,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,4514333,1,1,1,BOGOTA,1,2,1,51,⋯,NA,NA,NA,6703.343,NA,NA,NA,NA,1378973,6703.343


In [ ]:
#TODO: interpret

# 2. Age-Wage Analysis

## 2.1. Prep Data

In [23]:
# prep data
df <- df[df$age > 18 & df$ocu == 1, ]
names(df) <- make.names(names(df), unique = TRUE) # set df names

# drop missing wages, log transform
df <- df %>%
    filter(!is.na(y_ingLab_m), !is.na(age)) %>%
    mutate(log_wage = log(y_ingLab_m + 1))


## 2.2. Model Estimation

In [24]:
# fit OLS model
model <- lm(log_wage ~ age + I(age^2), data = df)
print(summary(model))


Call:
lm(formula = log_wage ~ age + I(age^2), data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7449 -0.3833 -0.1060  0.3007  3.8503 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.238e+01  7.335e-02  168.73   <2e-16 ***
age          8.509e-02  3.878e-03   21.94   <2e-16 ***
I(age^2)    -9.880e-04  4.773e-05  -20.70   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7423 on 9782 degrees of freedom
Multiple R-squared:  0.05007,	Adjusted R-squared:  0.04988 
F-statistic: 257.8 on 2 and 9782 DF,  p-value: < 2.2e-16



## 2.3. Bootstrap SE's

In [26]:
# define a function to apply bootstrap to regression estimates
boot_fn <- function(data, indices) {
    d <- data[indices, ]
    fit <- lm(log_wage ~ age + I(age^2), data = d)
    coef <- coef(fit)
    peak_age <- -coef[2] / (2 * coef[3])
    return(c(coef, peak_age = peak_age))
}

# define a function that calculates peak age (see model)
calc_peak_age <- function(data, indices) {
    d <- data[indices, ]
    model <- lm(log_wage ~ age + I(age^2), data = d)
    coef <- coef(model)
    peak_age <- -coef[2] / (2 * coef[3])
    return(peak_age)
}

# bootstrap
set.seed(555)
boot_results <- boot(data = df, statistic = boot_fn, R = 1000)

boot_se <- apply(boot_results$t, 2, sd)

# 2.4. Results

### Table

In [27]:
stargazer(model,
    se = list(boot_se), type = "text",
    title = "Age-Wage Profile Regression Results",
    column.labels = c("Coefficients", "Bootstrap SE")
)

# peak age & CI's
original_peak_age <- -coef(model)[2] / (2 * coef(model)[3])
ci_peak <- quantile(boot_results$t[, 4], c(0.025, 0.975))

cat("\nPeak Age Analysis:\n")
cat("Estimated Peak Age:", round(original_peak_age, 2), "\n")
cat(
    "95% CI for Peak Age:", round(ci_peak[1], 2),
    "to", round(ci_peak[2], 2), "\n\n"
)


Age-Wage Profile Regression Results
                        Dependent variable:    
                    ---------------------------
                             log_wage          
                           Coefficients        
-----------------------------------------------
age                          0.085***          
                              (0.005)          
                                               
I(age2)                      -0.001***         
                             (0.0001)          
                                               
Constant                     12.376***         
                              (0.084)          
                                               
-----------------------------------------------
Observations                   9,785           
R2                             0.050           
Adjusted R2                    0.050           
Residual Std. Error      0.742 (df = 9782)     
F Statistic          257.797*** (df = 2; 9782) 
Not

### Plot

In [43]:
# install.packages("IRkernel")
# IRkernel::installspec() 

In [44]:
temp_dir <- "~/R_temp"
dir.create(temp_dir, showWarnings = FALSE, recursive = TRUE)
Sys.setenv(TMPDIR = temp_dir)

png(filename = paste0(temp_dir, "/plot.png"))

In [52]:
new_data <- data.frame(age = seq(min(df$age), max(df$age), length.out = 100))
new_data$predicted_log_wage <- predict(model, newdata = new_data)

png(filename = paste0(temp_dir, "/plot.png"))

ggplot(df, aes(x = age, y = log_wage)) +
  geom_point(alpha = 0.1) +
  geom_line(data = new_data, aes(x = age, y = predicted_log_wage), color = "red") +
  geom_vline(
    xintercept = original_peak_age, color = "blue",
    linetype = "dashed") +
  geom_vline(xintercept = ci_peak[1], color = "green", linetype = "dashed") +
  geom_vline(xintercept = ci_peak[2], color = "green", linetype = "dashed") +
  labs(
    title = "Age-Wage Profile", x = "Age", y = "Log Wage",
    caption = "Blue dashed line: Estimated peak age\nGreen dashed lines: 95% CI") +
  theme_minimal()

#TODO: How to add plot :/

# 3. Gender Earnings GAP

## 3.1. Unconditional GAP

In [46]:
model_gap <- lm(log_wage ~ sex, data = df)
robust_se <- vcovHC(model_gap, type = "HC1") # heteroskedasticity robust SE
robust_summary <- coeftest(model_gap, vcov = robust_se)
print(robust_summary)


t test of coefficients:

             Estimate Std. Error   t value  Pr(>|t|)    
(Intercept) 13.948138   0.011551 1207.5540 < 2.2e-16 ***
sex          0.151050   0.015329    9.8536 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [ ]:
#TODO: interpret

## 3.2. Conditional GAP

### 3.2.1. Standard SE

In [47]:
# define control variables
controls <- c("age", "oficio", "p6210", "relab", "hoursWorkUsual")
## --> "p6220" refers to education attainment

# i) FWL: Standard SE's
formula_controls <- as.formula(paste(
    "log_wage ~",
    paste(controls, collapse = " + ")
))
formula_controls_sex <- as.formula(paste(
    "sex ~",
    paste(controls, collapse = " + ")
))

# partial out the controls
residuals_y <- feols(formula_controls, data = df)$residuals
residuals_sex <- feols(formula_controls_sex, data = df)$residuals

# regress residuals
fwl_model <- lm(residuals_y ~ residuals_sex)

# stats
fwl_coef <- coef(fwl_model)[2]
fwl_se <- sqrt(vcov(fwl_model)[2, 2])


### 3.2.2. Bootstrap SE

In [50]:
# recycle our old f
boot_fwl <- function(data, indices) {
    resid_y <- residuals_y[indices]
    resid_sex <- residuals_sex[indices]
    boot_model <- lm(resid_y ~ resid_sex)
    return(coef(boot_model)[2])
}

# bootstrap
set.seed(555)
boot_results <- boot(data = df, statistic = boot_fwl, R = 5000)

# bootstrap standard error
boot_se <- sd(boot_results$t)

### 3.2.3. Results

In [60]:
results_df <- data.frame(
    Term = c("Coefficient", "Standard Error (Normal)", "Standard Error (Bootstrap)"),
    Estimate = c(fwl_coef, fwl_se, boot_se)
)

stargazer(
  results_df,
  type = "text",
  summary = FALSE,
  rownames = FALSE,
  title = "Frisch-Waugh-Lovell Theorem Results",
  covariate.labels = c("Coefficient", "Statistic", "Standard Error (Bootstrap)"), # Keep original row labels
  coef = list(results_df$Estimate),
  digits = 5,
  notes = "Bootstrap standard errors are based on 1000 replications",
  out = "results_table.txt"
)

table_text <- readLines("results_table.txt")
table_text <- gsub("Standard Error \\(Normal\\)", "Statistic", table_text, fixed = FALSE)



Frisch-Waugh-Lovell Theorem Results
Coefficient                                Statistic     
---------------------------------------------------------
Coefficient                                 0.22470      
Standard Error (Normal)                     0.01312      
Standard Error (Bootstrap)                  0.01314      
---------------------------------------------------------
Bootstrap standard errors are based on 1000 replications 


### Plot 
Q: How?

In [ ]:
#TODO

In [ ]:
#TODO: interpret

# 4. Predicting Wage

## 4.1. Sample Split

In [61]:
set.seed(555)

n <- nrow(df)
train_indices <- sample(1:n, size = 0.7 * n, replace = FALSE)

train_data <- df[train_indices, ]  # Training set (70%)
test_data <- df[-train_indices, ]  # Testing set (30%)

cat("Training Set Size:", nrow(train_data), "\n")
cat("Testing Set Size:", nrow(test_data), "\n")

Training Set Size: 6849 
Testing Set Size: 2936 


## 4.2. Model Specification & Estimation